# W1M3 - ETL 프로세스 구현하기

In [6]:
import requests
from bs4 import BeautifulSoup

In [10]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29").text

In [11]:
soup = BeautifulSoup(html, 'html.parser')

In [12]:
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of countries by GDP (nominal) - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-

In [67]:
gdp = soup.find("table", {"class": 'wikitable'}).find('tbody').find_all('tr')[3:]
gdp_imf = [gdp_row.find_all('td')[1].text.strip() for gdp_row in gdp]
gdp_imf

['30,337,162',
 '19,534,894',
 '4,921,563',
 '4,389,326',
 '4,271,922',
 '3,730,261',
 '3,283,429',
 '2,459,597',
 '2,330,308',
 '2,307,162',
 '2,195,708',
 '1,947,133',
 '1,881,140',
 '1,827,576',
 '1,817,818',
 '1,492,618',
 '1,455,413',
 '1,272,960',
 '1,136,580',
 '999,604',
 '915,451',
 '814,438',
 '689,364',
 '638,780',
 '587,225',
 '574,201',
 '561,725',
 '568,567',
 '559,218',
 '550,905',
 '545,341',
 '506,466',
 '507,670',
 '488,250',
 '506,426',
 '481,856',
 '463,747',
 '431,228',
 '419,331',
 '422,057',
 '418,046',
 '406,202',
 '374,595',
 '362,240',
 '360,233',
 '345,874',
 '319,989',
 '319,934',
 '306,629',
 '294,898',
 '270,874',
 '265,171',
 '264,270',
 '262,924',
 '245,619',
 '226,217',
 '194,955',
 '189,827',
 '168,597',
 '161,950',
 '—',
 '145,031',
 '142,617',
 '126,238',
 '121,425',
 '120,971',
 '116,321',
 '113,286',
 '112,653',
 '112,369',
 '109,993',
 '108,425',
 '106,327',
 '95,149',
 '91,210',
 '89,665',
 '87,347',
 '86,993',
 '74,846',
 '83,883',
 '82,789',
 '

In [59]:
soup.find("table", {"class": 'wikitable'}).find('tbody').find_all('tr')[2:]

[<tr class="static-row-header" style="font-weight:bold;">
 <td style="text-align:left"><span class="flagicon" style="padding-left:25px;"> </span>World</td>
 <td>110,047,109</td>
 <td>2024</td>
 <td>105,435,540</td>
 <td>2023</td>
 <td>100,834,796</td>
 <td>2022
 </td></tr>,
 <tr>
 <td style="text-align:left"><span class="flagicon" style="display:inline-block;width:25px;text-align:left"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></span></span></span> <a class="mw-redirect" href="/wiki/GDP_of

In [34]:
tag_list = []
for tr_tag in soup.find('tr', {'class': 'static-row-header'}) :
    print(tr_tag)



<th rowspan="2">Country/Territory
</th>


<th colspan="2"><a href="/wiki/International_Monetary_Fund" title="International Monetary Fund">IMF</a><sup class="reference" id="cite_ref-GDP_IMF_2-2"><a href="#cite_note-GDP_IMF-2"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-15"><a href="#cite_note-15"><span class="cite-bracket">[</span>13<span class="cite-bracket">]</span></a></sup>
</th>


<th colspan="2"><a href="/wiki/World_Bank" title="World Bank">World Bank</a><sup class="reference" id="cite_ref-16"><a href="#cite_note-16"><span class="cite-bracket">[</span>14<span class="cite-bracket">]</span></a></sup>
</th>


<th colspan="2"><a href="/wiki/United_Nations" title="United Nations">United Nations</a><sup class="reference" id="cite_ref-UN_17-0"><a href="#cite_note-UN-17"><span class="cite-bracket">[</span>15<span class="cite-bracket">]</span></a></sup>
</th>


In [22]:
for i in soup.find("table", {"class": 'wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter'}) :
    print(i)

TypeError: 'NoneType' object is not iterable

In [19]:
table = soup.find(class_='wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter')
if table:
    tbody = table.find('tbody')
    if tbody:
        for tr_tag in tbody.find_all('tr'):
            ...


In [25]:
print(soup.find("table", {"class": 'wikitable'}))

<table class="wikitable sortable sticky-header-multi static-row-numbers" style="text-align:right">
<caption>GDP (million US$) by country
</caption>
<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th rowspan="2">Country/Territory
</th>
<th colspan="2"><a href="/wiki/International_Monetary_Fund" title="International Monetary Fund">IMF</a><sup class="reference" id="cite_ref-GDP_IMF_2-2"><a href="#cite_note-GDP_IMF-2"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-15"><a href="#cite_note-15"><span class="cite-bracket">[</span>13<span class="cite-bracket">]</span></a></sup>
</th>
<th colspan="2"><a href="/wiki/World_Bank" title="World Bank">World Bank</a><sup class="reference" id="cite_ref-16"><a href="#cite_note-16"><span class="cite-bracket">[</span>14<span class="cite-bracket">]</span></a></sup>
</th>
<th colspan="2"><a href="/wiki/United_Nations" title="United Nations">United Na